## Imports

In [1]:
# standard imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import time
import pickle

# webscraping
import requests
from bs4 import BeautifulSoup

# stocks
import yfinance as yf
import cvxopt as opt
from cvxopt import blas, solvers
import cufflinks
import mplfinance as mpf

# import plotly.plotly as py
import plotly.tools as tls
from plotly.graph_objs import *
solvers.options['show_progress'] = False

# ignore warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
# import sys
# !{sys.executable} -m pip install mplfinance

## Basic Goal

Plan: 
1. Data gathering
      - Randomly select 30 stocks from the S&P 500 
      - Get data for each of the stocks from the past 3 years using yfinance.  
      - Get the daily Fama-French factors from the Kenneth French website.
2. Implement the PCA Markowitz portfolio optimization
      - PCA on normalized returns
      - Check if PC1 is significant using Tracy-Widow
      - Get the portfolio that corresponds to PC1
3. Implement the Fama-French three-factor model
      - For each stock, run the standard time series regression for the Fama-French model. 
      - Get the covariance matrix from the residuals 
4. Compare the 2 portfolios against the efficient frontier

### Data Gathering

In [3]:
# getting the stocks
headers = {
    'User-Agent': (
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 '
        '(KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    )
}

response = requests.get(
    "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies",
    headers=headers
)
response.raise_for_status()
tables = pd.read_html(response.text)

if len(tables) > 0:
    stocks_df = tables[0]

# randomly selecting 30 stocks
random_stocks = stocks_df['Symbol'].sample(n=30, random_state=42)

In [4]:
# getting closing prices for the 30 stocks with batching
start_date = '2021-01-01'
end_date = '2024-12-31'

def download_stocks_in_batches(tickers, batch_size=5, delay=1):
    """
    Download stock data in batches to avoid rate limiting
    """
    all_data = {}
    
    for i in range(0, len(tickers), batch_size):
        batch = tickers[i:i + batch_size]
        print(f"Downloading batch {i//batch_size + 1}: {batch}")
        
        try:
            # Download the batch
            batch_data = yf.download(
                batch,
                start=start_date,
                end=end_date,
                progress=False
            )
            
            # Extract closing prices for this batch
            if not batch_data.empty and 'Close' in batch_data.columns:
                closes = batch_data['Close']
                # Handle single ticker case (returns Series instead of DataFrame)
                if isinstance(closes, pd.Series):
                    all_data[batch[0]] = closes
                else:
                    for ticker in closes.columns:
                        all_data[ticker] = closes[ticker]
                print(f"Successfully downloaded {len(batch)} stocks")
            else:
                print(f"No data returned for batch: {batch}")
            
        except Exception as e:
            print(f"Error downloading batch {batch}: {e}")
        
        # Add delay between batches to avoid rate limiting
        if i + batch_size < len(tickers):
            print(f"Waiting {delay} seconds before next batch...")
            time.sleep(delay)
    
    # Combine all data into a single DataFrame
    if all_data:
        return pd.DataFrame(all_data)
    else:
        return pd.DataFrame()

# Download in batches of 5 stocks with 1-second delay
closing_df = download_stocks_in_batches(
    random_stocks.tolist(), 
    batch_size=5, 
    delay=15
)

YF.download() has changed argument auto_adjust default to True



5 Failed downloads:
['CINF', 'DHI', 'K', 'LIN', 'BR']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


No data returned for batch: ['K', 'BR', 'LIN', 'DHI', 'CINF']
Waiting 15 seconds before next batch...



5 Failed downloads:
['LHX', 'BA', 'O', 'IDXX', 'GLW']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


No data returned for batch: ['LHX', 'O', 'GLW', 'BA', 'IDXX']
Waiting 15 seconds before next batch...


KeyboardInterrupt: 

This code above was generated with ChatGPT.

In [ ]:
if not closing_df.empty:
    closing_df.to_pickle('closing prices.pkl')

closing_df.head(5)

In [12]:
# getting fama french factors
ff_data = pd.read_csv(
    'ff_factors.csv', 
    header=2
)
ff_data.rename(columns={'Unnamed: 0': 'date'}, inplace=True)
ff_data = ff_data.iloc[:422]

# getting dates
ff_data['date'] = ff_data['date'].astype(str).str.strip()

for idx in range(len(ff_data['date'])):
    date = ff_data.iloc[idx, 0]
    ff_data.iloc[idx, 0] = date[:4] + '-' + date[-2:]

ff_data['date'] = pd.to_datetime(ff_data['date'])
ff_data.set_index('date', inplace=True)
ff_data = ff_data.loc['2021':'2024-12']

,Mkt-RF,SMB,HML,RF
date,,,,
2021-01-01,-0.54,2.97,1.01,0.00
2021-02-01,2.88,0.43,7.46,0.00
2021-03-01,3.12,-2.34,5.58,0.00
2021-04-01,4.49,-1.04,-1.97,0.00
2021-05-01,1.65,-0.44,4.14,0.00
2021-06-01,1.01,-0.29,-5.23,0.00
2021-07-01,1.04,-2.31,-1.67,0.00
2021-08-01,2.42,0.10,-1.71,0.00
2021-09-01,-3.81,0.77,3.86,0.00


### Implement the PCA Markowitz portfolio optimization

### Implement the Fama-French three-factor model

### Compare the 2 portfolios against the efficient frontier

## Intermediate Goal

Plan:
1. Implement the RIE Markowitz portfolio optimization
      - Estimate the covariance matrix by doing RIE on the log returns
2. Check which eigenvectors are significant using Marchenco-Pastur and pick one to use for the portfolio (this should be the second largest eigenvalue).
3. Compare its fit to the efficient frontier.

### Implement the RIE Markowitz portfolio optimization

### Check which eigenvectors are significant using Marchenco-Pastur and pick one to use for the portfolio

### Compare its fit to the efficient frontier.

## Advanced Goal

Plan:
As per the study done by Molero-Gonzales et al., (2023), the steps we will follow are the ff:
1. Apply the Fama-French 3 Factor Model on the log returns matrix.
2. Turn the coefficients for each of the factors for each stock in the portfolio into a matrix.
3. Apply RMT.
      - Eigenvalue decomposition
      - Identify the significant eigenvectors using the Tracy-Widom distribution.
4. Create a portfolio out of the significant factors and the residuals.
5. Compare its performance against the matrix in the Basic Goal section to predict stock risk.

### Apply the Fama-French 3 Factor Model on the log returns matrix.

### Turn the coefficients for each of the factors for each stock in the portfolio into a matrix.

### Apply RMT

### Create a portfolio out of the significant factors and the residuals.

### Compare its performance against the matrix in the Basic Goal section to predict stock risk.